In [ ]:
import requests
import bs4
import re
import os
import pandas as pd
import json

In [ ]:
def get_cricbuzz_link(text):
    try:
        url = 'https://google.com/search?q=' + text
        request_result=requests.get( url )
        soup = bs4.BeautifulSoup(request_result.text,
                             "html.parser")
        heading_object=soup.find_all( 'a' )
        req=''
        for info in heading_object:
            link=info.get('href')
            x=re.search(".*https://www.cricbuzz.com/profiles/.*",link)
            if x:
                req=link
        return re.findall('https://www.cricbuzz.com/profiles/[0-9]+/[a-z,-]+',req)[0]
    except:
        return "Failed"

In [ ]:
def get_espn_link(text):
    try:
        url = 'https://google.com/search?q=' + text
        request_result=requests.get( url )
        soup = bs4.BeautifulSoup(request_result.text,
                             "html.parser")
        heading_object=soup.find_all( 'a' )
        #print(heading_object)
        req='1'
        for info in heading_object:
            link=info.get('href')
            x=re.search(".*https://www.espncricinfo.com/player/.*",link)
            #print(x)
            if x:
                req=link
        if req=='1':
            return "Failed"
        return re.findall('https://www.espncricinfo.com/player/[0-9,a-z,-]+',req)[0]
    except:
        return "Failed"

In [ ]:
def get_cricbuzz_data(url):
    try:
        req=requests.get(url)
        soup = bs4.BeautifulSoup(req.text,
                                 "html.parser")
        div=soup.find_all("div", class_="cb-col cb-col-60 cb-lst-itm-sm")
        data=[]
        posts=["WK-Batsman","Batsman","Bowling Allrounder","Batting Allrounder","Bowler","--"]
        c=0
        for i in div:
            if str(i.string).strip() in posts:
                c=1
            if c==1:
                data.append(str(i.string).strip())
        data=data[:len(data)-1]
        if len(data)!=3:
            data.append("null")
        return data
    except:
        return "Failed"

In [ ]:
def get_espn_data(url):
    try:
        req=requests.get(url)
        soup = bs4.BeautifulSoup(req.text,
                                 "html.parser")
        div=soup.find_all("h5")
        #print(div)
        data=[]
        posts=["Wicketkeeper Batter","Middle order Batter","Allrounder","Top order Batter","Bowler","Batting Allrounder","Opening Batter"]
        c=0
        for i in div:
            data.append(str(i.string).strip())
            if str(i.string).strip() in posts:
                c=len(data)
        return [data[c-1],data[c-3],"null" if data[c-2]=="Wicketkeeper" else data[c-2]]
    except:
        return "Failed"

In [ ]:
odi_dir='odis_json'
t20_dir='t20s_json'
players=[]

In [ ]:
try:
    for filename in os.listdir(odi_dir):
        json_file=open(os.path.join(odi_dir,filename))
        file=json.load(json_file)
        for j in file["info"]["players"]:
              for k in file["info"]["players"][j]:
                    if not ([k,j]) in players:
                        players.append([k,j])
except:
    print(len(players))

2170


In [ ]:
data={"Name":[],
      "Team":[],
      "Position":[],
      "Batting":[],
      "Bowling":[]}
k=0
c=[]

In [ ]:
for i in players:
    k=k+1
    try:
        res=get_cricbuzz_data(get_cricbuzz_link(i[0]))
        if(res=="Failed"):
            res=get_cricbuzz_data(get_cricbuzz_link(i[0]+i[1]))
        if res=="Failed":
            res=get_cricbuzz_data(get_cricbuzz_link(i[0]+i[1]+" cricketer"))
        if res=="Failed":
            res=get_cricbuzz_data(get_cricbuzz_link(i[0]+" cricbuzz.com"))
        if res=="Failed":
            res=get_espn_data(get_espn_link(i[0]))
        if(res=="Failed"):
            res=get_espn_data(get_espn_link(i[0]+i[1]))
        if res=="Failed":
            res=get_espn_data(get_espn_link(i[0]+i[1]+" cricketer"))
        if res=="Failed":
            res=get_espn_data(get_espn_link(i[0]+" espncricinfo.com"))
        if not (len(res) ==3):
            raise Exception("INVALID")
        data["Name"].append(i[0])
        data["Team"].append(i[1])
        data["Position"].append(res[0])
        data["Batting"].append(res[1])
        data["Bowling"].append(res[2])
    except:
        c.append(i)
    if k%200==0:
        print("*")
        print(len(c))
        print(len(data["Name"]))

*
7
193
*
26
374
*
42
558
*
74
726
*
114
886
*
163
1037
*
165
1235
*
174
1426
*
187
1613
*
235
1765


In [ ]:
df=pd.DataFrame(data)

In [ ]:
df.to_csv("Players.csv")

In [ ]:
i=['M Prasidh Krishna', 'India']
res=get_cricbuzz_data(get_cricbuzz_link(i[0]))
if(res=="Failed"):
    res=get_cricbuzz_data(get_cricbuzz_link(i[0]+i[1]))
if res=="Failed":
    res=get_cricbuzz_data(get_cricbuzz_link(i[0]+i[1]+" cricketer"))
if res=="Failed":
    res=get_cricbuzz_data(get_cricbuzz_link(i[0]+" cricbuzz.com"))
if res=="Failed":
    res=get_espn_data(get_espn_link(i[0]))
if(res=="Failed"):
    res=get_espn_data(get_espn_link(i[0]+i[1]))
if res=="Failed":
    res=get_espn_data(get_espn_link(i[0]+i[1]+" cricketer"))
if res=="Failed":
    res=get_espn_data(get_espn_link(i[0]+" espncricinfo.com"))

In [ ]:
res

['--', 'Bowler', 'Right Handed Bat', 'Right-arm fast-medium', 'null']

In [ ]:
get_cricbuzz_data(get_cricbuzz_link("MK Pandey cricketer"))

['Batsman', 'Right Handed Bat', 'Right-arm medium']

In [ ]:
len(c)

249

In [ ]:
players_left={
    "Name":[],
    "Team":[]
}

In [ ]:
for i in c:
    players_left["Name"].append(i[0])
    players_left["Team"].append(i[1])

In [ ]:
df2=pd.DataFrame(players_left)
df.to_csv("Players_left.csv")

In [ ]:
df2

,Name,Team
0,CD de Lange,Scotland
1,Ehsan Khan,Hong Kong
2,Waqas Khan,Hong Kong
3,Tanveer Ahmed,Hong Kong
4,Taskin Ahmed,Bangladesh
...,...,...
244,Sami Aslam,Pakistan
245,RV Kalpana,India
246,GA Guy,New Zealand
247,L Sreekumar,United Arab Emirates
